In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from kaggle_environments.envs.rps.utils import get_score
from kaggle_environments import make

In [ ]:
from scipy import stats

In [ ]:
rock = 0
paper = 1
scissors = 2

num_games = 18
outcomes = np.zeros(num_games)

for i in range(num_games):
    # let it be random
    outcomes[i] = np.random.choice([rock, paper, scissors])

In [ ]:
values, counts = np.unique(outcomes, return_counts=True)
probs = counts / len(outcomes)
probs

In [ ]:
reverse = np.ones(len(values)) - probs
reverse = reverse / np.sum(reverse)

In [ ]:
values

In [ ]:
np.random.choice(values, p=reverse)

In [ ]:
def pdf(outcomes):
    val, cnt = np.unique(outcomes, return_counts=True)
    prop = cnt / len(outcomes)
    return (val, prop)

In [ ]:
np.histogram(outcomes, bins=[rock, paper, scissors])

In [ ]:
val, prop = pdf(outcomes[0:4])
prop

In [ ]:
plt.bar(*pdf(outcomes[0:4]))
plt.ylabel("Probability")
plt.xlabel("Outcome")
plt.show()
plt.close()

In [ ]:
%%writefile nash_equilibrium_defeat_agent.py

import random
import numpy as np

memory = {}

def beat_nash_equilibrium_agent(observation, configuration):
    signs = list(range(0, configuration.signs))
    global memory
    if observation.step > 0:
        # Save observation in memory
        memory[observation.lastOpponentAction] += 1
        # Calculate PMF of previous actions
        counts = list(memory.values())
        freqs = counts / np.sum(counts)
        # Calculate PMF of the next action
        # It is reverse of previous action PMF as it should try to restore Equilibrium
        probs = np.ones(configuration.signs) - freqs
        probs = probs / np.sum(probs)
        # Predict next action
        predict = np.random.choice(signs, p=probs)
        # Return our response to that action
        return int(predict % configuration.signs)
    else:
        memory = dict.fromkeys(signs, 0)
        return random.randrange(0, configuration.signs)

In [ ]:
from kaggle_environments.envs.rps.agents import *

def randomAgent(obs, config):
    return random.randint(0, 2)

In [ ]:
env = make('rps', debug = True)

In [ ]:
# Pass in your agent to evaluate
def evaluate(agent):
    
    pool = [
        randomAgent,
        randomAgent,
        randomAgent,
        randomAgent,
        randomAgent,
        
        rock,
        paper,
        scissors,
        copy_opponent,
        reactionary,
        counter_reactionary,
        statistical,
        randomAgent,
    ]
    overall = 0

    for opponent in pool:
        env.reset()
        env.run([agent, opponent])
        json = env.toJSON()
        rewards = json['rewards']

        opponentName = str(opponent).split()[1].capitalize()

        padding = ''; length = len(str(rewards))
        if length < 15: padding = ' ' * (15 - length)

        print(f'Rewards: {rewards}{padding}  Agent vs {opponentName}')
        overall += rewards[0]
    
    overall /= len(pool)
    print(f'Overall Score: {overall}')
    
    return overall

In [ ]:
score = evaluate('/kaggle/working/nash_equilibrium_defeat_agent.py')
if score <= 0:
    print('\nU MIGHT WANNA FIX SOME THINGS BUDDY')
else:
    print('\nITS GREAT! NOW CHUCK IT INTO THE LEADERBOARD')